In [4]:
with open ('sites.txt') as f:
    lines = f.readlines()
    lines = [line.rstrip() for line in lines]
    first_site = lines[0]
f.close()

print(first_site)

https://scholarslab.lib.virginia.edu/events/


In [ ]:
# Custom HTML Parsing as solution vs. LLM Text Extraction w/ HTML Filtering

In [ ]:
# TODO: extract the correct information to fit the schema
